In [1]:
# !pip install transformers -q

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K

from transformers import *

In [3]:
import torch

In [2]:
tf.__version__

'2.2.0'

In [4]:
from transformers import TFBertModel, BertModel, BertConfig

In [6]:
bert = 'bert-base-cased'

In [7]:
tokenizer = BertTokenizer.from_pretrained(bert)


In [72]:
config = BertConfig()
# config.vocab_size = 28996 # pretrained size
config.output_hidden_states = True
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [37]:
bert_model = BertModel.from_pretrained(bert, config=config)

INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/bert-base-cased-pytorch_model.bin from cache at /Users/rowancassius/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2


In [56]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /Users/rowancassius/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


AssertionError: 

In [67]:
tokenizer.tokenize('Hello my name is Rowan at ')

['Hello', 'my', 'name', 'is', 'Rowan', 'at', 'AM', '##M']

In [39]:
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])

In [40]:
tokens_tensor

tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]])

In [41]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor #.to('cuda')
segments_tensors = segments_tensors  #.to('cuda')
# model  #.to('cuda')

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/rowancassius/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin from cache at /Users/rowancassius/.cache/torch/tra

In [42]:
# Predict hidden states features for each layer
with torch.no_grad():
    # See the models docstrings for the detail of the inputs
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    # Transformers models always output tuples.
    # See the models docstrings for the detail of all the outputs
    # In our case, the first element is the hidden state of the last layer of the Bert model
    encoded_layers = outputs[0]
# We have encoded our input sequence in a FloatTensor of shape (batch size, sequence length, model hidden dimension)

assert tuple(encoded_layers.shape) == (1, len(indexed_tokens), model.config.hidden_size)



In [47]:
outputs[0].shape

torch.Size([1, 14, 768])

In [48]:
outputs[1].shape

torch.Size([1, 768])

In [18]:
encoded_layers.shape

torch.Size([1, 14, 768])

In [256]:
bert_model = TFBertModel.from_pretrained(bert, config=config)

input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')

# sequence_output, pooled_output, (hidden_states), (attentions)
context_embedding = bert_model(input_ids)[0]

decoder = tf.keras.layers.LSTM(4, return_sequences=True)(context_embedding)
# decoder, final_memory_state, final_carry_state = tf.keras.layers.LSTM(
#     4, return_sequences=True, return_state=True)(context_embedding)

decoded = tf.keras.layers.TimeDistributed(
    tf.keras.layers.Dense(tokenizer.vocab_size + 1, activation='softmax'))(decoder)

model = tf.keras.Model(inputs=input_ids, outputs=decoded)

model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_token (InputLayer)     [(None, 128)]             0         
_________________________________________________________________
tf_bert_model_9 (TFBertModel ((None, 128, 768), (None, 108310272 
_________________________________________________________________
lstm_17 (LSTM)               (None, 128, 4)            12368     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 128, 28997)        144985    
Total params: 108,467,625
Trainable params: 108,467,625
Non-trainable params: 0
_________________________________________________________________


In [235]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [236]:
sentences = ["hello my name is Rowan"]
tokens = tokenizer.encode_plus(sentences)
tokens

{'input_ids': [101, 100, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [237]:
prediction = model.predict(tokens['input_ids'])

In [238]:
prediction.shape

(3, 1, 28997)

In [225]:
prediction

array([[[3.4487486e-05, 3.4515844e-05, 3.4374112e-05, ...,
         3.4576788e-05, 3.4483124e-05, 3.4550892e-05]],

       [[3.4421519e-05, 3.4500699e-05, 3.4452059e-05, ...,
         3.4421271e-05, 3.4399454e-05, 3.4439017e-05]],

       [[3.4485805e-05, 3.4514011e-05, 3.4375415e-05, ...,
         3.4573593e-05, 3.4483448e-05, 3.4547324e-05]]], dtype=float32)

In [239]:
model.fit(x=tokens['input_ids'], y = tokens['input_ids'])

1/1 [==============================] - 0s 5ms/step - loss: 10.2753


## Generate Sequence Attempt

In [258]:
type(bert_model.get_output_embeddings())

NoneType

In [259]:
bert_model_lm = TFBertForMaskedLM.from_pretrained(bert, config=config)

In [262]:
type(bert_model.get_output_embeddings())

NoneType

## Encoder Decoder Attempt

In [263]:
from transformers import EncoderDecoderModel, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased') # initialize Bert2Bert

# forward
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)

# training
loss, outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, lm_labels=input_ids)[:2]

# generation
generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)

In [268]:
tokenizer.decode(input_ids[0])

'[CLS] hello, my dog is cute [SEP]'

In [264]:
generated

tensor([[   0, 6506, 6506, 6506, 6506, 6506, 2666, 2666, 2666, 2666, 2666, 4618,
         4618, 4618, 4618, 4618, 4618, 4618, 4618, 4618]])

In [265]:
tokenizer.decode(generated[0])

'[PAD] leon leon leon leon leonieieieieie shall shall shall shall shall shall shall shall shall'